<a href="https://colab.research.google.com/github/ValentineFil/-Projects_in_Pytorch/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22A_Dive_into_Natural_Language_Processing_Lesson_2%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A Dive into Natural Language Processing.

#### Text preprocessing. Embeddings.

- Познакомимся с основными инструментами и библиотеками для обработки текста
- Познакомимся с эмбеддингами
- Сделаем семантический поиск

## Знакомимся с Colab

Colab — среда для проведения экспериментов, позволяющая использовать ресурсы Google. Код пишется на python'е.

![](https://i.imgur.com/F4mV2Kf.png)

### Код

Пара shortcut'ов:


|shortcut| details|
|-|-|
|Сtrl + Enter | Выполнить текущую ячейку | 
|Сtrl + F8 | Выполнить все, что выше | 
|Сtrl + B | Добавить ячейку снизу |
|Сtrl + A | Добавить ячейку сверху |
|Сtrl + M + M | Превратить в ячейку с текстом |
|Сtrl + M + Y | Превратить в ячейку с кодом |
|Сtrl + M + D | Удалить ячейку |
|Сtrl + ?| Комментирование |

In [ ]:
text = "Some text"

print(text)

Some text


### Shell

Через **!** можно запускать консольные команды.

Например, **!nvidia-smi** покажет, какая видеокарта есть в текущем окружении (если GPU выбран в настройках).

In [ ]:
!python --version

Python 3.7.12


In [ ]:
!nvidia-smi

Fri Oct 15 15:54:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.74       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Видеокарты

Бесплатно выдается Tesla K80, может попасться Tesla T4. На тарифе Colab Pro (\$10) — Tesla T4 и Tesla P100. На Colab Pro+ (\$50) — Tesla V100 и Tesla A100 (40 Gb).

**Ограничения**:

- На длительность отдельной сессии
- На общее использование

Надо закрывать неиспользуемые вкладки или останавливать сессию руками. Если использовать слишком много (по несколько суток с перезапуском), то можно увидеть что-то подобное:

![](https://i.imgur.com/PGXvYM9.png)


### Google Drive

Можно подключать Google Drive, копировать с него данные и сохранять на нем результаты расчетов.

In [ ]:
!cat /etc/os-release

NAME="Ubuntu"
VERSION="18.04.5 LTS (Bionic Beaver)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 18.04.5 LTS"
VERSION_ID="18.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=bionic
UBUNTU_CODENAME=bionic


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/

In [ ]:
!mkdir /content/drive/MyDrive/nlp_course

mkdir: cannot create directory ‘/content/drive/MyDrive/nlp_course’: File exists


In [ ]:
!echo "NLP course is here." > /content/drive/MyDrive/nlp_course/test.txt

In [ ]:
!cat /content/drive/MyDrive/nlp_course/test.txt

Можно загружать файлы из интернета

In [ ]:
!wget https://images.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png -O img.png

In [ ]:
from IPython.display import Image
Image('img.png')

Можно загружать шаренные файлы с Google Drive.

In [ ]:
!gdown --id 11Q2g2aA-n7fYK95XEB3d6m0CWKSt6rob

Downloading...
From: https://drive.google.com/uc?id=11Q2g2aA-n7fYK95XEB3d6m0CWKSt6rob
To: /content/quora_question_pairs_rus.csv.zip
100% 26.7M/26.7M [00:00<00:00, 125MB/s] 


### Установка зависимостей

Установка пакетов производится через менеджер пакетов **pip**.

Так как при запуске сессии поднимается образ среды, специально собранный для работы с данными, то многие пакеты уже установлены.

In [ ]:
import nltk

Если какого-то пакета нет, то его можно установить.

In [ ]:
!pip install razdel pymorphy2 pymystem3

import razdel

     |████████████████████████████████| 55 kB 3.7 MB/s 
     |████████████████████████████████| 8.2 MB 25.8 MB/s 


Если пакет есть, но нужна другая версия, то его можно переустановить. После этого может потребоваться перезапуск среды выполнения. При этом текущие переменные сотрутся, файлы останутся.

In [ ]:
import gensim

print(gensim.__version__)

In [ ]:
!pip install -U gensim==3.8.0

     |████████████████████████████████| 24.2 MB 1.3 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


## Предобработка текста

### Токенизация

Процесс разбиения текста на токены. Например, на слова.

Токенизировать можно разными способами, давайте посмотрим на самый простой — разбиение регулярками.

In [ ]:
from nltk.tokenize import WordPunctTokenizer

tokenizer = WordPunctTokenizer()

In [ ]:
text = "В каком американском штате 1/2 находится Большой Каньон..."

print(tokenizer.tokenize(text))

['В', 'каком', 'американском', 'штате', '1', '/', '2', 'находится', 'Большой', 'Каньон', '...']


In [ ]:
from razdel import tokenize, sentenize

In [ ]:
list(tokenize(text))

[Substring(0, 1, 'В'),
 Substring(2, 7, 'каком'),
 Substring(8, 20, 'американском'),
 Substring(21, 26, 'штате'),
 Substring(27, 30, '1/2'),
 Substring(31, 40, 'находится'),
 Substring(41, 48, 'Большой'),
 Substring(49, 55, 'Каньон'),
 Substring(55, 58, '...')]

### Лемматизация

Процесс приведения слова к его нормальной форме (лемме).

In [ ]:
# import pymystem3

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemmatize(text):
    words = text.split()
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return ' '.join(res)

In [ ]:
text = "В каком американском штате находится Большой Каньон?"

print(lemmatize(text))

в какой американский штат находиться большой каньон?


### Удаление stopwords

In [ ]:
import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from string import punctuation

russian_stopwords = stopwords.words("russian")

def remove_stopwords(text):
  tokens = tokenizer.tokenize(text)
  res = [token for token in tokens if token.lower() not in russian_stopwords\
         and token not in punctuation]

  return ' '.join(res)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
russian_stopwords

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [ ]:
text = "В каком американском штате находится Большой Каньон?"

print(remove_stopwords(text))

каком американском штате находится Большой Каньон


In [ ]:
def preprocess_text(text):
  text = remove_stopwords(text)
  text = lemmatize(text)

  return text

In [ ]:
text = "В каком американском штате находится Большой Каньон?"

tokenizer.tokenize(preprocess_text(text))

['какой', 'американский', 'штат', 'находиться', 'большой', 'каньон']

## Датасет

In [ ]:
!git clone https://github.com/vladislavneon/RuBQ.git

fatal: destination path 'RuBQ' already exists and is not an empty directory.


In [ ]:
import json

filepath = "/content/RuBQ/RuBQ_2.0/RuBQ_2.0_test.json"

with open(filepath, mode="r", encoding="utf-8") as input:
  lines = input.readlines()

# print(lines)

res_json = json.loads(''.join(lines))

In [ ]:
lines[:40]

['[\n',
 '    {\n',
 '        "uid": 0,\n',
 '        "question_text": "Что может вызвать цунами?",\n',
 '        "query": "SELECT ?answer \\nWHERE {\\n  wd:Q8070 wdt:P828 ?answer\\n}",\n',
 '        "answer_text": "Землетрясение",\n',
 '        "question_uris": [\n',
 '            "http://www.wikidata.org/entity/Q8070"\n',
 '        ],\n',
 '        "question_props": [\n',
 '            "wdt:P828"\n',
 '        ],\n',
 '        "answers": [\n',
 '            {\n',
 '                "type": "uri",\n',
 '                "label": "землетрясение",\n',
 '                "value": "http://www.wikidata.org/entity/Q7944",\n',
 '                "wd_names": {\n',
 '                    "ru": [\n',
 '                        "землетрясение",\n',
 '                        "җир тетрәве"\n',
 '                    ],\n',
 '                    "en": [\n',
 '                        "seism",\n',
 '                        "earthquake",\n',
 '                        "seismic activity",\n',
 '               

In [ ]:
for q in res_json:
  print(q)
  break

{'uid': 0, 'question_text': 'Что может вызвать цунами?', 'query': 'SELECT ?answer \nWHERE {\n  wd:Q8070 wdt:P828 ?answer\n}', 'answer_text': 'Землетрясение', 'question_uris': ['http://www.wikidata.org/entity/Q8070'], 'question_props': ['wdt:P828'], 'answers': [{'type': 'uri', 'label': 'землетрясение', 'value': 'http://www.wikidata.org/entity/Q7944', 'wd_names': {'ru': ['землетрясение', 'җир тетрәве'], 'en': ['seism', 'earthquake', 'seismic activity', 'fore shocks', 'tremor', 'earthquakes', 'earth quake', 'earthtemblor', 'foreshock', 'aftershock', 'quake', 'temblor', 'earth temblor', 'foreshocks', 'after shocks', 'earth quakes', 'after shock', 'earthtremor', 'convulsion', 'earth tremor', 'shock', 'fore shock', 'aftershocks']}, 'wp_names': ['землетрясениям']}, {'type': 'uri', 'label': 'метеорит', 'value': 'http://www.wikidata.org/entity/Q60186', 'wd_names': {'ru': ['метеорит', 'Метеориты', 'Аэролиты'], 'en': ['shooting star', 'meteorite']}, 'wp_names': []}, {'type': 'uri', 'label': 'опол

In [ ]:
dataset_temp = [x["question_text"] for x in res_json]

In [ ]:
dataset_temp[:10]

['Что может вызвать цунами?',
 'Кто написал роман «Хижина дяди Тома»?',
 'Кто автор пьесы «Ромео и Джульетта»?',
 'Как называется столица Румынии?',
 'На каком инструменте играл Джимми Хендрикс?',
 'Какой стране принадлежит остров Таити?',
 'Кто создал Зимний дворец в Санкт-Петербурге?',
 'Кто режиссер фильма "Бриллиантовая рука"?',
 'Какой стране принадлежат Канарские острова?',
 'Какой географический объект называется Лимпопо?']

In [ ]:
len(dataset_temp)

2330

In [ ]:
#kaggle
!unzip quora_question_pairs_rus.csv.zip

Archive:  quora_question_pairs_rus.csv.zip
  inflating: quora_question_pairs_rus.csv  


In [ ]:
import pandas as pd

quora_df = pd.read_csv('quora_question_pairs_rus.csv')

In [ ]:
quora_df.head(20)

,Unnamed: 0,question1,question2,is_duplicate
0,0,Какова история кохинор кох-и-ноор-бриллиант,"что произойдет, если правительство Индии украд...",0
1,1,как я могу увеличить скорость моего интернет-с...,как повысить скорость интернета путем взлома ч...,0
2,2,"почему я мысленно очень одинок, как я могу это...","найти остаток, когда математика 23 ^ 24 матема...",0
3,3,которые растворяют в воде быстро сахарную соль...,какая рыба выживет в соленой воде,0
4,4,астрология: я - луна-колпачок из козерога и кр...,Я тройная луна-козерог и восхождение в козерог...,1
5,5,я должен купить tiago,что делает детей активными и далеки от телефон...,0
6,6,как я могу быть хорошим геологом?,"что я должен делать, чтобы быть великим геологом?",1
7,7,когда вы используете вместо,когда вы используете вместо,0
8,8,компания motorola: могу ли я взломать мой чарт...,как я могу взломать motorola dcx3400 для беспл...,0
9,9,метод определения разделения прорезей с исполь...,что некоторые технические специалисты могут ра...,0


In [ ]:
dataset_temp.extend(quora_df["question1"].values[:50000])

In [ ]:
dataset_temp.extend(quora_df["question2"].values[:50000])

In [ ]:
len(dataset_temp)

102330

In [ ]:
dataset = []

for x in dataset_temp:
  if isinstance(x, str) and x.strip():
    dataset.append(x)

In [ ]:
len(dataset)

102325

In [ ]:
import random

random.choices(dataset, k=10)

['насколько вероятна мировая война в будущем',
 'вы можете увидеть, кто просматривает ваши видеоролики с instagram',
 'я должен купить роскошный автомобиль lamborghini aventador, который стоит 400 000',
 'почему бы не работать в google',
 'каковы три разных типа протистов',
 'привет, мне сейчас 24, и у меня появился интерес к велоспорту, это означает, что на велосипеде на велосипеде, это слишком поздно, чтобы начать',
 'это saab gripen так же хорошо, как тайфун европы',
 'если хартинский клинтон победит на этих выборах, какова будет ее политика в отношении Индии',
 'кто бог, кто-нибудь видел бога',
 'примерно, сколько стоит chevy g10 van весенний модель 1993 года, если возможно']

## Embeddings

Существует большое количество предтренированных embedding'ов. Можно натренировать на википедии свои при помощи библиотеки _gensim_. В ней тоже есть предтренированные модели.

- https://radimrehurek.com/gensim/

Есть модели для русского языка RusVectōrēs.

- https://rusvectores.org/ru/models/


Для русского NLP есть хороший набор библиотек [natasha](https://github.com/natasha/natasha). В этом проекте есть много чего, возьмем из него эмбеддинги.

- https://github.com/natasha/natasha
- https://habr.com/ru/post/516098/

### Gensim

In [ ]:
import gensim.downloader as api

model = api.load("glove-wiki-gigaword-50")

In [ ]:
model.most_similar(positive=['woman', 'king'], negative=['man'])

In [ ]:
model.most_similar('tea')

### RusVectores

In [ ]:
#https://github.com/RaRe-Technologies/gensim-data

model_ru = api.load("word2vec-ruscorpora-300")

In [ ]:
model_ru.most_similar(positive=['женщина_NOUN', 'король_NOUN'], negative=['мужчина_NOUN'])

In [ ]:
model_ru.most_similar('город_NOUN')

### Natasha

In [ ]:
!pip install navec

!wget https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar

--2021-10-15 15:58:54--  https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53012480 (51M) [application/x-tar]
Saving to: ‘navec_hudlit_v1_12B_500K_300d_100q.tar’

navec_hudlit_v1_12B 100%[===================>]  50.56M  40.4MB/s    in 1.3s    

2021-10-15 15:58:55 (40.4 MB/s) - ‘navec_hudlit_v1_12B_500K_300d_100q.tar’ saved [53012480/53012480]



In [ ]:
from navec import Navec

path = 'navec_hudlit_v1_12B_500K_300d_100q.tar'
navec = Navec.load(path)

In [ ]:
navec["текст"]

## Считаем близость

Есть эмбеддинги слов, хотим получить эмбеддинги для предложений.

In [ ]:
import numpy as np

In [ ]:
def embed(text):
  res = []

  tokens = tokenizer.tokenize(str(text).lower())

  for token in tokens:
    if token in navec:
      res.append(navec[token])

  if not res:
    res.append(navec['<unk>'])

  return np.mean(res, axis=0)

In [ ]:
embed("Вот такой вот текст")

In [ ]:
embedded_data = [embed(x) for x in dataset]

In [ ]:
def cosine(u, v):
  res = np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
  return res

In [ ]:
def get_result(text):
  query = embed(text)

  cosines = [cosine(x, query) for x in embedded_data]

  for i in np.argpartition(cosines, -10)[-10:]:
    print(dataset[i])

  # np.argsort(cosines)


In [ ]:
text = "Как сдать икзамен?"

get_result(text)

как вы можете сдать свой чек в любом банке
как пройти экзамен 1z0 - 425?
как пройти экзамен ccsk
я хочу подготовиться к upsc, у меня есть 6 месяцев бесплатно, а затем год обучения engg, как я могу это сделать, если я планирую сдавать экзамен в 2018 году
как я раскошу экзамены ecet
как я могу легко изучить экзамен 1z0 - 548
как я могу сдать экзамен ccna с первого раза
Я только что закончил мой выпуск в механическом engg из pune univ, поэтому, чтобы получить работу, что мне делать, какие экзамены на выпускные экзамены для выпускников
как сделать 500000 как cpa
вы пишете взять или сдавать экзамен?


## Датасет с опечатками

In [ ]:
!wget https://github-typo-corpus.s3.amazonaws.com/data/github-typo-corpus.v1.0.0.jsonl.gz -O typos.gz

In [ ]:
!gunzip /content/typos.gz

In [ ]:
import pandas as pd

data_df = pd.read_json('typos', lines=True)

In [ ]:
data_df.head()

In [ ]:
!git clone https://github.com/dkulagin/kartaslov.git

In [ ]:
import pandas as pd

typos_df = pd.read_csv('/content/kartaslov/dataset/orfo_and_typos/orfo_and_typos.L1_5.csv', delimiter=';')

In [ ]:
typos_df.head()

In [ ]:
typos_dict = dict()

for index, row in typos_df.iterrows():
  if not row['CORRECT'] in typos_dict:
    typos_dict[row['CORRECT']] = row['MISTAKE']

In [ ]:
len(typos_dict)

In [ ]:
[x for x in typos_dict.items()][:20]

In [ ]:
dataset[:10]

In [ ]:
for i,line in enumerate(dataset):
  words = line.split()
  for i,x in enumerate(words):
    if x in typos_dict:
      words[i] = typos_dict[x]
  dataset[i] = ' '.join(words)


In [ ]:
dataset[:10]

## fastText

In [ ]:
#facebook
# !pip install fasttext
# import fasttext.util

In [ ]:
# fasttext.util.download_model('ru', if_exists='ignore')
# ft = fasttext.load_model('cc.ru.300.bin')

In [ ]:
#deeppavlov
# !wget http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_nltk_wordpunct_tokenize/ft_native_300_ru_wiki_lenta_nltk_wordpunct_tokenize.vec

In [ ]:
!wget http://vectors.nlpl.eu/repository/20/181.zip

--2021-10-15 16:02:27--  http://vectors.nlpl.eu/repository/20/181.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2622716250 (2.4G) [application/zip]
Saving to: ‘181.zip’

181.zip             100%[===================>]   2.44G  34.2MB/s    in 32s     

2021-10-15 16:02:59 (77.8 MB/s) - ‘181.zip’ saved [2622716250/2622716250]



In [ ]:
!unzip 181.zip

Archive:  181.zip
  inflating: meta.json               
  inflating: model.model             
  inflating: model.model.vectors_ngrams.npy  
  inflating: model.model.vectors.npy  
  inflating: model.model.vectors_vocab.npy  
  inflating: README                  


In [ ]:
import gensim

model = gensim.models.KeyedVectors.load('model.model')

In [ ]:
vec = model.get_vector("сабака")

print(vec)


In [ ]:
def embed_fasttext(text):
  res = []

  tokens = tokenizer.tokenize(str(text).lower())

  for token in tokens:
      res.append(model.get_vector(token))

  return np.mean(res, axis=0)

In [ ]:
embedded_data = [embed_fasttext(x) for x in dataset]

In [ ]:
query = embed_fasttext("Как бросто сдать икзамены?")

cosines = [cosine(x, query) for x in embedded_data]

for i in np.argpartition(cosines, -10)[-10:]:
  print(dataset[i])

вы можете сдавать экзамены после окончания средней школы
у меня есть лицензия на страхование жизни и здоровья в штате Ута, это означает, что я могу продать ее только в штате Юта или мне нужно сдать экзамен в каждом штате
как взломать ipcc-экзамен
как я могу сдать экзамен ccna с первого раза
как мне взломать dse 2017 вступительный экзамен
как мне взломать dse 2016 вступительный экзамен
как вы можете сдать свой чек в любом банке
что лучше делать после сдачи экзамена
что лучше делать после сдачи экзамена
вы пишете взять или сдавать экзамен?
